In [8]:
import random
import numpy as np
import pandas as pd
import time
import chess
from teradataml import *

In [9]:
# Connect with Teradata Database
eng = create_context("192.168.56.1", "val", "val")
conn = get_connection()
replay_buffer_table = "replay_buffer"
replay_buffer_staging_table = "replay_buffer_stage"

In [10]:
tdf = DataFrame(replay_buffer_staging_table)
batch_df = pd.DataFrame({
    'game_id': [],
    'ply': [],
    'FEN': [],
    'move': [],
    'reward': [],
})

In [11]:
def play_game(in_board, game_id, training=None):
    
    # Initialize the starting variables.
    sequence_no = 0
    reward = 0
    
    game_df = pd.DataFrame({
        'game_id': [],
        'ply': [],
        'FEN': [],
        'move': [],
        'reward': [],
    })
    board = chess.Board()
    if in_board is not None:
        board = in_board

    # Launch into the key loop.
    while not board.is_checkmate() and not board.is_stalemate() and not board.is_insufficient_material() and not board.can_claim_draw() and not board.is_seventyfive_moves() and not board.is_fivefold_repetition():
        if training and sequence_no == 0:
            fen = board.fen()
            spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
            opening_fen = fen[:spaces_pos[3]]
            
            ply_array = np.array([game_id, sequence_no, opening_fen, "", 0])
            new_row = pd.DataFrame([ply_array], columns=game_df.columns)
            game_df = pd.concat([game_df, new_row], ignore_index=True)

        sequence_no += 1
        all_moves = str(board.legal_moves)
        start_pos = [pos for pos, char in enumerate(all_moves) if char == '(']
        end_pos = [pos for pos, char in enumerate(all_moves) if char == ')']
        all_moves_string= all_moves[start_pos[0]+1:end_pos[0]]
        all_moves_string = all_moves_string.replace(" ", "")
        all_moves_array = all_moves_string.split(',')
        next_move = random.choice(all_moves_array)
        board.push_san(next_move)

        if training:
            fen = board.fen()
            spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
            trimmed_fen = fen[:spaces_pos[3]]
            

            ply_array = np.array([game_id, sequence_no, trimmed_fen, next_move, 0])
            new_row = pd.DataFrame([ply_array], columns=game_df.columns)
            game_df = pd.concat([game_df, new_row], ignore_index=True)

    if board.is_checkmate():
        if board.outcome().winner:
            reward = 1
        else:
            reward = -1
    
    reward_array = np.full(sequence_no+1, reward)
    game_df['reward'] = reward_array

    return game_df, reward

In [12]:
num_games = 5000
batch_size = 100
batch_counter = 0
out_reward = 0
start_time = time.time()

# Set the game_id by looking up the last used game_id.
game_id_cursor = execute_sql("SELECT MAX(game_id) FROM " + replay_buffer_table +";")
in_game_id = int(game_id_cursor.fetchall()[0][0]) + 1  # There has to be a better way to do this.

for counter in range(num_games):
    batch_counter = batch_counter + 1
    out_replay_buffer_df, out_reward = play_game(None, in_game_id, training=True)
    print("Reward for game " + str(in_game_id) + " was " + str(out_reward) + ".")
   
    # Increment the game_id
    in_game_id = in_game_id + 1

    # Append the game dataframe to the batch dataframe.
    batch_df = pd.concat([batch_df, out_replay_buffer_df], ignore_index=True)
    
    if batch_counter == batch_size or counter == num_games:
        
        # Load the batch to the database.
        fastload(df = batch_df, table_name = replay_buffer_staging_table, if_exists = 'append')
        
        # Empty the batch dataframe.
        batch_df = pd.DataFrame({
            'game_id': [],
            'ply': [],
            'FEN': [],
            'move': [],
            'reward': [],
        })
        
        # Reset the batch counter.
        batch_counter = 0
        end_time = time.time()
        batch_duration = end_time-start_time
        print(str(counter+1) + " of " + str(num_games) + " completed. Last batch duration was " + str(batch_duration) + " with average game duration of " + str(batch_duration/batch_size) + ".")
        start_time = time.time()


Reward for game 5501 was 0.
Reward for game 5502 was 0.
Reward for game 5503 was 0.
Reward for game 5504 was 0.
Reward for game 5505 was 0.
Reward for game 5506 was 0.
Reward for game 5507 was 0.
Reward for game 5508 was 0.
Reward for game 5509 was 0.
Reward for game 5510 was 0.
Reward for game 5511 was 0.
Reward for game 5512 was 0.
Reward for game 5513 was 0.
Reward for game 5514 was 0.
Reward for game 5515 was 0.
Reward for game 5516 was 0.
Reward for game 5517 was 0.
Reward for game 5518 was 0.
Reward for game 5519 was 0.
Reward for game 5520 was 0.
Reward for game 5521 was 0.
Reward for game 5522 was 0.
Reward for game 5523 was 1.
Reward for game 5524 was -1.
Reward for game 5525 was 0.
Reward for game 5526 was 0.
Reward for game 5527 was 0.
Reward for game 5528 was 0.
Reward for game 5529 was 0.
Reward for game 5530 was 0.
Reward for game 5531 was 0.
Reward for game 5532 was -1.
Reward for game 5533 was 0.
Reward for game 5534 was 0.
Reward for game 5535 was 1.
Reward for game 55

In [13]:
execute_sql("CALL UpdateProcessBatch()")

TeradataCursor uRowsHandle=248 bClosed=False

In [14]:
conn.close
remove_context()

True